In [ ]:
import pickle
from functools import partial

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from joblib import delayed, Parallel

from utils import bracket_0, rfc_W
from common import Bracket
from seeding import Seeding
from mcmc import MetropolisHastingsBracket

In [ ]:
bracket = Bracket.NaiveBracket(bracket_0(), rfc_W())
bracket.score()

In [ ]:
mhb = MetropolisHastingsBracket(bracket_0(), win_matrix=rfc_W(), simulate_anneal=True)
mhb.run(3000)[-1].score()

In [ ]:
W = rfc_W()
b = bracket_0()
_i, _j = np.meshgrid(range(64), range(64))
mat = np.zeros((64, 64))
for (ni, i) in tqdm(list(enumerate(_i))):
    for nj, j in enumerate(_j):
        mat[ni, nj] = W[b[_i[ni, nj]], b[_j[ni, nj]]]
df = pd.DataFrame(mat, columns=[i.name for i in b], index=[i.name + "_l" for i in b])
df.style.background_gradient(cmap='Blues')

In [ ]:
d = df.T.mean().sort_values()
t = [list(filter(lambda x: x.name == i.split("_")[0], b))[0] for i in d.index.tolist()]
s = Seeding(t, W)
print(s)
print(s.mean_variance(iters=100000))
print(s.find_maximimum_likelihood_bracket(iters=3000, verbose=True).score())
print(s.mlb.depth_error())
print(s.mlb.outcome_error(s))

In [ ]:
ncaa_seed = Seeding(Seeding.inverse_arrange(bracket_0()), W)
print(ncaa_seed)
print(ncaa_seed.mean_variance(iters=100000))
print(ncaa_seed.find_maximimum_likelihood_bracket(iters=3000, verbose=True).score())
print(ncaa_seed.mlb.depth_error())
print(ncaa_seed.mlb.outcome_error(ncaa_seed))

In [ ]:
TOP_K = 100
with open("./seeding_optim_for_use.pkl", "rb") as doc:
    _optimal_seed = pickle.load(doc)
optimal_seed = []
scores = []
for s in _optimal_seed:
    if s._score not in scores and s._score is not None:
        optimal_seed.append(s)
        scores.append(s._score)
print(sorted(scores, reverse=True))
plt.plot([i._score for i in _optimal_seed])
plt.show()
optimal_seed = [i[-1] for i in sorted([(i._score if i._score is not None else -1000000, n, i) for n, i in enumerate(optimal_seed)])[-TOP_K:]]
for seed in optimal_seed:
    seed._score = None
    seed.win_matrix = W
def mv(x):
    return x.mean_variance(iters=100000)
scores = Parallel(n_jobs=8)(delayed(partial(x.mean_variance, iters=100000, verbose=False))() for x in tqdm(optimal_seed))#scores = process_map(mv, optimal_seed)#.append(seed.mean_variance(iters=100000))
optimal_seed = optimal_seed[np.argmax(scores)]

In [ ]:
optimal_seed.win_matrix = W
print(optimal_seed)
print(optimal_seed.mean_variance(iters=100000))
print(optimal_seed.find_maximimum_likelihood_bracket(iters=3000, verbose=True).score())
print(optimal_seed.mlb.depth_error())
print(optimal_seed.mlb.outcome_error(optimal_seed))